# Social Network Analysis 


We're analyzing Jordan’s road network as a graph to:

1. Find important intersections using centrality metrics.

2. These metrics help answer:

        - Which intersections are well connected?
        - Which are closest to all others?
        - Which act as critical bridges in traffic flow?


In [ ]:
 # pip install osmnx networkx pandas matplotlib geopandas


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing necessary libraries 

import osmnx as ox # Access and process real-world street networks from OpenStreetMap
import networkx as nx # flexible, extensible, and powerful network analysis tool
import pandas as pd # data analysis and manipulation tool
import matplotlib.pyplot as plt # visualizations

In [2]:
# Step 1: Load the Jordan road network
place_name = "Jordan"
G = ox.graph_from_place(place_name, network_type='all', simplify=True)



#ox.graph_from_place(...) to downloads the road network 
#Nodes = intersections, Edges = road segments.

""" Parameters:
    network_type='all' : Includes all types of paths: roads, footpaths, service lanes.... 
    simplify=True: Merges intermediate nodes (like bend points) so each road segment is a clean edge between intersections.
Makes the graph more usable for centrality and routing analysis."""



/Users/masaaladwan/.pyenv/versions/3.11.6/lib/python3.11/site-packages/osmnx/_overpass.py:267: UserWarning: This area is 48 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


KeyboardInterrupt: 

In [13]:
G.size()

641595

Then that means your road network has over 640,000 nodes (intersections)  which is very large , lets focus on amman 
I've been running the code for more than 12 hours, so for this project i will simplfy it more 


In [3]:
G_amman = ox.graph_from_place("Amman, Jordan", network_type='drive', simplify=True)

In [4]:
G_amman.size()

156456

Thats so much better, still it will be slow for betweenes centrality 

In [10]:
# Step 2: Convert to undirected 
G_undirected = G.to_undirected()


# why u may ask ? becaues the method i want to use is much faster when its used to process undirected graphs
# Assuming symmetrical movement is often OK for large-scale analysis
# If you care about direction (like for routing or smth), keep the original directed graph G 

In [5]:
G_undirected_amman = G_amman.to_undirected()
G_undirected_amman.size()

89336

In [11]:
# Step 3: Compute centralities
degree_centrality = nx.degree_centrality(G_undirected)

# Degree centrality counts direct connections only it says how many roads meet at a junction.
# But it says nothing about: How central the location is to the entire network ?  Whether it lies on important travel routes between cities or not 
# basically it only cares about the number of roads that connect to a node, not how important that node is to the network as a whole., NO GLOBAL IMPORTANCE
# so we use other centrality measures to get a more complete picture of a node's importance in the network.


In [6]:
degree_centrality_amman = nx.degree_centrality(G_undirected_amman)


In [12]:
closeness_centrality = nx.closeness_centrality(G_undirected, distance='length')

# Useful for identifying the most accessible locations ,helps planners find where to place services like emergency stations..... 
# How close it is to all other intersections, based on the shortest path distances in the network.

KeyboardInterrupt: 

In [ ]:
closeness_centrality_amman = nx.closeness_centrality(G_undirected_amman, distance='length')


In [ ]:
betweenness_centrality = nx.betweenness_centrality(G_undirected, weight='length', normalized=True)

In [ ]:
betweenness_centrality_amman = nx.betweenness_centrality(G_undirected_amman, weight='length', normalized=True, k=1000)

In [ ]:
# Step 4: Assign attributes
nx.set_node_attributes(G_undirected, degree_centrality, 'degree_centrality')



In [ ]:
nx.set_node_attributes(G_undirected_amman, degree_centrality_amman, 'degree_centrality')



In [ ]:
nx.set_node_attributes(G_undirected, closeness_centrality, 'closeness_centrality')


In [ ]:
nx.set_node_attributes(G_undirected_amman, closeness_centrality_amman, 'closeness_centrality')


In [ ]:
nx.set_node_attributes(G_undirected, betweenness_centrality, 'betweenness_centrality', k=1000, normalized=True)

In [ ]:
nx.set_node_attributes(G_undirected_amman, betweenness_centrality_amman, 'betweenness_centrality', k=1000, normalized=True)

In [ ]:

# Step 5: Visualize betweenness
nc = ox.plot.get_node_colors_by_attr(G_undirected, attr='betweenness_centrality', cmap='plasma')


In [ ]:
nc = ox.plot.get_node_colors_by_attr(G_undirected_amman, attr='betweenness_centrality', cmap='plasma')


In [ ]:
fig, ax = ox.plot_graph(G_undirected, node_color=nc, node_size=10, edge_linewidth=0.5, bgcolor='white')


In [ ]:

# Step 6: Export results (NEW API)
nodes, edges = ox.graph_to_gdfs(G_undirected)
nodes.to_csv("jordan_nodes_centrality.csv")
edges.to_csv("jordan_edges.csv")
nodes.to_file("jordan_nodes_centrality.shp")
edges.to_file("jordan_edges.shp")


In [ ]:

# Step 7: Print top 5 by betweenness
top_btwn = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
for node, score in top_btwn:
    print(f"Node {node}: Betweenness = {score}, Lat = {G_undirected.nodes[node]['y']}, Lon = {G_undirected.nodes[node]['x']}")